# Numeric Representations:  In-Class Exercises

[Open in Colab](https://colab.research.google.com/github/sdrangan/hwdesign/blob/main/units/unit02_numbers/numbers_inclass.ipynb)

To motivate fixed point representations, in this exercise, let's consider a problem of translating a basic floating point operation to a limited number of values.  We will see some ways to do this in the class, but let's think about how we would do this simple problem to get you thinking about what is involved.

Consider a simple a quadratic function:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nx = 1000
x = np.linspace(-4, 4, nx)
y = 3.2 - 0.7 * (x**2)

plt.plot(x, y)
plt.grid()
plt.xlabel('x')
plt.ylabel('y')


Now suppose we have to perform the same computation in a small number, say `b=8` bits.
Think of a way to do the following:

* **Encode** each of the values `x` to `b` bits
* **Compute** `y` from the encoded values of `x`.  You can only use a `b x b` integer multiplier and adders.  Also, you must store the final value of `y` back in `b` bits
* **Decode** the `b` bit values of `y` back to floating point numbers
* Measure the **approximation error**  of using just `b` bits.

-21